In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import random
import pickle

**goal: 1000 datapoints, 15 features**

counties to scrape:
 - Alameda County, CA
 - Santa Clara County, CA
 - San Mateo County, CA
 - San Francisco County, CA

In [1]:
al_county = 'alameda-county-ca/'
sc_county = 'santa-clara-county-ca/'
sf_county = 'san-francisco-county-ca/'
sm_county = 'san-mateo-county-ca/'

In [39]:
url = 'https://www.compass.com/for-rent/' + sm_county
response = requests.get(url)
print(response)
search_page = response.text
search_page_soup = BeautifulSoup(search_page, 'lxml')

<Response [200]>


In [40]:
end_of_browse_idx = search_page.find(' Browse ')+ len(' Browse')
num_listings_div_20_ceiling = -(-int(search_page[end_of_browse_idx:end_of_browse_idx + 5])// 20)
print('number of listings in query: ', search_page[end_of_browse_idx:end_of_browse_idx + 5])
num_listings_div_20_ceiling

number of listings in query:   323 


17

In [45]:
# last two tags are not property listings so they are excluded
response = requests.get(url)
print(response)
search_page = response.text
search_page_soup = BeautifulSoup(search_page, 'lxml')

url_details = []

start = 0
for page in range(1, num_listings_div_20_ceiling+1):
    print('\npage ', page)
    for link in search_page_soup.find_all('a'):
        if str(link.get('href'))[:9] == '/listing/':
            print(link.get('href'))
            url_details.append(link.get('href'))
    start += 20
    print(start)
    start_str = str(start)
    url = url + '/start=' + start_str
    response = requests.get(url)
    print(response)
    search_page = response.text
    search_page_soup = BeautifulSoup(search_page, 'lxml')
    time.sleep(random.randint(1, 4))

<Response [200]>

page  1
20
<Response [200]>

page  2
/listing/937-rollins-road-unit-4-burlingame-ca-94010/634659368654306417/
/listing/105-west-santa-inez-avenue-hillsborough-ca-94010/610996815347122873/
/listing/444-richmond-drive-unit-2-millbrae-ca-94030/683346322090523785/
/listing/2622-martinez-drive-burlingame-ca-94010/639917476723268017/
/listing/50-shady-lane-hillsborough-ca-94010/637659540088880745/
/listing/2755-marlborough-avenue-redwood-city-ca-94063/663034586991305249/
/listing/901-holly-road-belmont-ca-94002/677530750404568225/
/listing/1313-hoover-street-menlo-park-ca-94025/674017864532705849/
/listing/3337-brittan-avenue-unit-8-san-carlos-ca-94070/654440844381116977/
/listing/436-laurel-street-unit-2-san-carlos-ca-94070/615278578697314009/
/listing/2074-potomac-way-san-mateo-ca-94403/663895280376891457/
/listing/27-engle-road-san-mateo-ca-94402/652244651256943585/
/listing/307-haight-street-menlo-park-ca-94025/670375821327173449/
/listing/15-martin-lane-woodside-ca-940

In [47]:
print('number of urls: ', len(url_details))

number of urls:  318


---

**part ii:** loop through all links acquired and obtain the soup object for each rental listing

---

In [48]:
rental_url_base = 'https://www.compass.com'
urls = [rental_url_base + details for details in url_details]

#responses = [requests.get(url) for url in urls]

responses = []

for url in urls:
    response = requests.get(url)
    responses.append(response)
    print(len(responses))
    time.sleep(random.randint(1,3))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [49]:
# compile text from each listing into a list
listings = [response.text for response in responses]

<span style="color:red">!!</span> update county name before pickling

In [50]:
county_name = 'san_mateo'
date = datetime.today().strftime('%y%m%d')

In [51]:
with open(county_name +'_county_listings_' + date + '.pickle', 'wb') as to_write:
    pickle.dump(listings, to_write)